In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_datagen=ImageDataGenerator(rescale=1/255.0,shear_range=0.2,zoom_range=0.2,brightness_range=[0.5,1.5],validation_split=0.2,horizontal_flip=True)

In [3]:
train_set=train_datagen.flow_from_directory(r"C:\Users\pramy\OneDrive\Desktop\mask_pred\dataset",target_size=(128,128),batch_size=32,class_mode='binary',subset='training')

Found 352 images belonging to 2 classes.


In [4]:
validation_set=train_datagen.flow_from_directory(r"C:\Users\pramy\OneDrive\Desktop\mask_pred\dataset",target_size=(128,128),batch_size=32,class_mode='binary',subset='validation')

Found 88 images belonging to 2 classes.


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [6]:
model=Sequential()

In [7]:
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

C:\Users\pramy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.add(Flatten())

In [9]:
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation='sigmoid'))

In [10]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(train_set,epochs=20,validation_data=validation_set)

C:\Users\pramy\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.5199 - loss: 0.6864 - val_accuracy: 0.8523 - val_loss: 0.5658
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8080 - loss: 0.5614 - val_accuracy: 0.8523 - val_loss: 0.4303
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 22s 2s/step - accuracy: 0.8343 - loss: 0.4478 - val_accuracy: 0.8864 - val_loss: 0.3266
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.8715 - loss: 0.3195 - val_accuracy: 0.8523 - val_loss: 0.3470
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.8703 - loss: 0.3291 - val_accuracy: 0.9091 - val_loss: 0.2412
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.9198 - loss: 0.2624 - val_accuracy: 0.9091 - val_loss: 0.2420
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.9173 - loss: 0.2198 - val_accuracy: 0.9091 - val_loss: 0.3387
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 20s 2s/step - accuracy: 0.8948 - loss: 0.3000 - val_accuracy: 0.9205 - val_loss:

In [16]:
model.save('mask_detector_model.keras')

In [17]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [18]:
model = load_model('mask_detector_model.keras')
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap=cv2.VideoCapture(0)
while True:
    r,frame=cap.read()
    if r!=True:
        break
        
    frame=cv2.flip(frame,1)
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) 
    face=face_cascade.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=8)
    for(x,y,w,h) in face:
        face_img=frame[y:y+h,x:x+w]
        face_img=cv2.resize(face_img,(128,128))
        face_img=cv2.cvtColor(face_img,cv2.COLOR_BGR2RGB)
        face_img=face_img/255.0
        face_img=np.expand_dims(face_img,axis=0)

        prediction=model.predict(face_img)[0][0]

        if prediction>0.5:
            label='No Mask'
            color=(0,0,255)
        else:
            label='Mask'
            color=(0,255,0)
            
        cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
        cv2.putText(frame,label,(x, y - 10),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,2)


    cv2.imshow('Mask Detection',frame)
    #Exit on 'q' key
    if cv2.waitKey(1) & 0xff==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━